In [ ]:
!pip install plotly
!pip install streamlit

In [55]:
import streamlit as st
import random
import plotly.express as px
import pandas as pd

In [56]:
df = pd.read_excel('Article.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Squad                  98 non-null     object 
 1   Competition            98 non-null     object 
 2   Avg Age                98 non-null     float64
 3   Avg Possession%        98 non-null     float64
 4   Field_tilt%            98 non-null     float64
 5   PPDA                   98 non-null     float64
 6   OPPDA                  98 non-null     float64
 7   Attempted passes/90    98 non-null     float64
 8   CrsPA/90               98 non-null     float64
 9   SCA from dead balls    98 non-null     float64
 10  Tackles def3rd/90      98 non-null     float64
 11  Tackles mid3rd/90      98 non-null     float64
 12  Tackles final3rd/90    98 non-null     float64
 13  Touches/90             98 non-null     float64
 14  Dribbles Attempted/90  98 non-null     float64
 15  Aerials 

In [ ]:
#from IPython.core.debugger import decorate_fn_with_doc
df_style_of_play = pd.DataFrame()
df_style_of_play['Team'] = df['Squad']
df_style_of_play['Pressing Intensity'] = df['PPDA']
df_style_of_play['Pressing Resistance'] = df['OPPDA']
df_style_of_play['Possession'] = df['Avg Possession%']
df_style_of_play['Field tilt %'] = df['Field_tilt%']
df_style_of_play['Tempo'] = df['Attempted passes/90']*100/df['Touches/90']
df_style_of_play['Directness'] = df['PrgDist']*100/df['TotDist']
df['%tackles_def3rd'] = df['Tackles def3rd/90'] / (df['Tackles def3rd/90'] + df['Tackles mid3rd/90'] + df['Tackles final3rd/90']) 
df['%tackles_mid3rd'] = df['Tackles mid3rd/90'] / (df['Tackles def3rd/90'] + df['Tackles mid3rd/90'] + df['Tackles final3rd/90']) 
df['%tackles_final3rd'] = df['Tackles mid3rd/90'] / (df['Tackles def3rd/90'] + df['Tackles mid3rd/90'] + df['Tackles final3rd/90']) 
df_style_of_play['Defensive height'] = (df['%tackles_def3rd']*16.5*1.1) + (df['%tackles_mid3rd']*49.5*1.1) + (df['%tackles_final3rd']*82.5*1.1) 
df_style_of_play['Crossing'] = df['CrsPA/90']
df_style_of_play['Air duels'] = df['Aerials Lost'] + df['Aerials Won'] 
df_style_of_play['Set Pieces'] = df['SCA from dead balls']
df_style_of_play['Dribbling'] = df['Dribbles Attempted/90'] / df['Touches/90']
df_style_of_play.head()
#df_style_of_play[df_style_of_play['Team']=='Liverpool']

In [69]:
# normalization
df_style_of_play['Dribbling'] = (df_style_of_play['Dribbling']*100) / (df_style_of_play['Dribbling'].max())
df_style_of_play['Set Pieces'] = (df_style_of_play['Set Pieces']*100) / (df_style_of_play['Set Pieces'].max())
df_style_of_play['Air duels'] = (df_style_of_play['Air duels']*100) / (df_style_of_play['Air duels'].max())
df_style_of_play['Crossing'] = (df_style_of_play['Crossing']*100) / (df_style_of_play['Crossing'].max())
df_style_of_play['Air duels'] = (df_style_of_play['Air duels']*100) / (df_style_of_play['Air duels'].max())
df_style_of_play['Pressing Resistance'] = (df_style_of_play['Pressing Resistance']*100) / (df_style_of_play['Pressing Resistance'].max())
df_style_of_play['Pressing Intensity'] = (df_style_of_play['Pressing Intensity'].min()*100) / (df_style_of_play['Pressing Intensity'])

#print(max(df_style_of_play['Dribbling']))

In [ ]:
df_style_of_play.head(30)

In [76]:
st.sidebar.header("Please Filter Here:")
team = st.sidebar.selectbox(
    "Select or write team name:\nTeam names are sorted alphabetically.",
    options=df_style_of_play['Team'].unique()
)
df_style_of_play1 = df_style_of_play.query("Team == @team")

In [84]:
def radar_chart(team_name):
    #df_style_of_play = df_style_of_play[df_style_of_play['Team']==team_name]
    df = pd.DataFrame(dict(
    r=[df_style_of_play1['Pressing Intensity'],
       df_style_of_play1['Pressing Resistance'],
       df_style_of_play1['Possession'],
       df_style_of_play1['Field tilt %'],
       df_style_of_play1['Tempo'],
       df_style_of_play1['Directness'],
       df_style_of_play1['Defensive height'],
       df_style_of_play1['Crossing'],
       df_style_of_play1['Air duels'],
       df_style_of_play1['Set Pieces'],
       df_style_of_play1['Dribbling']
       ],
    theta=['Pressing Intensity',	'Pressing Resistance',	'Possession',	'Field tilt %',	'Tempo',	'Directness',	'Defensive height',	'Crossing',	'Air duels',	'Set Pieces',	'Dribbling']))
    fig = px.line_polar(df, r='r', theta='theta', line_close=True)
    st.write(fig)

In [85]:
if __name__ == '__main__':
    radar_chart(team_name)